In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt

In [ ]:
print(tf.version.VERSION)

# Set-Up Dataset

In [ ]:
labels = ['A-BLUE', 'A-RED', 'B-BLUE', 'B-RED']
img_cols = 640
img_rows = 360
row_shape = 0
col_shape = 1

def read_img(path, img, class_num, data):
    try:
        img_arr = cv2.imread(os.path.join(path, img)) # read image with opencv
        if img_arr.shape[row_shape] == img_rows and img_arr.shape[col_shape] == img_cols:
            data.append([img_arr, class_num]) # add to final data list
        else:
            print('File at', os.path.join(path,img), 'does not have proper size')
            print('Expected ({}x{}}) but got ({}x{})'.format(img_rows,img_cols,img_arr.rows,img_arr.cols))
    except Exception as e:
        print(e)

def read_data(data_dir, train_set_size_per_cat, val_set_size_per_cat):
    train_data = []
    val_data = []
    
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        
        images = os.listdir(path) # list of images in directory
        training_images = random.sample(images, train_set_size_per_cat) # select random images for training
        images = [image for image in images if image not in training_images] # remove selected images
        validation_images = random.sample(images, val_set_size_per_cat) # select random validation images
        
        # read the training images
        for img in training_images:
            read_img(path, img, class_num, train_data)
                
        # read the validation images
        for img in validation_images:
            read_img(path, img, class_num, val_data)
    
    return np.array(train_data), np.array(val_data)

In [ ]:
train, val = read_data('input', 100, 40)

x_train = []
y_train = []
x_val = []
y_val = []

for feature, label in train:
    x_train.append(feature)
    y_train.append(label)

for feature, label in val:
    x_val.append(feature)
    y_val.append(label)

# normalize
x_train = np.array(x_train) / 255
x_val = np.array(x_val) / 255

y_train = np.array(y_train)
y_val = np.array(y_val)

y_train = keras.utils.to_categorical(y_train)
y_val = keras.utils.to_categorical(y_val)

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center             =False,  # set input mean to 0 over the dataset
        samplewise_center              =False,  # set each sample mean to 0
        featurewise_std_normalization  =False,  # divide inputs by std of the dataset
        samplewise_std_normalization   =False,  # divide each input by its std
        zca_whitening                  =False,  # apply ZCA whitening
        rotation_range                 =3,      # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range                     =0.1,    # Randomly zoom image 
        width_shift_range              =0.1,    # randomly shift images horizontally (fraction of total width)
        height_shift_range             =0.1,    # randomly shift images vertically (fraction of total height)
        horizontal_flip                =False,  # randomly flip images
        vertical_flip                  =False)  # randomly flip images

datagen.fit(x_train)

# Create and Train Model

In [ ]:
def create_model(rows, cols):
    model = tf.keras.models.Sequential([
        keras.layers.Conv2D(32, 3, padding="same", activation="relu", input_shape=(rows,cols,3)),
        keras.layers.MaxPool2D(),
        keras.layers.Conv2D(32, 3, padding="same", activation="relu"),
        keras.layers.MaxPool2D(),
        keras.layers.Conv2D(64, 3, padding="same", activation="relu"),
        keras.layers.MaxPool2D(),
        keras.layers.Dropout(0.2),
        keras.layers.Flatten(),
        keras.layers.Dense(112, activation="relu"),
        keras.layers.Dense(56, activation="relu"),
        keras.layers.Dense(56, activation="relu"),
        keras.layers.Dense(4, activation="softmax"),
    ])
    opt = keras.optimizers.Adam(lr=0.001)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = create_model(img_rows, img_cols)
history = model.fit(x_train, y_train, epochs=25, validation_data=(x_val, y_val), shuffle=True, verbose=2)

In [ ]:
loss, acc = model.evaluate(x_val, y_val, verbose=2)

# Inference Function

In [ ]:
def inference(model, img, expected):
    res = model.predict(img)
    real = expected
    
    int_res_one_hot = [int(round(r)) for r in res[0]]
    int_real_one_hot = [int(round(r)) for r in real]
    #print('Encoded Result: {} | Encoded Expected: {}'.format(int_res_one_hot, int_real_one_hot))
    
    res_idx = -1
    real_idx = -1
    res_ = 'NONE'
    real_ = 'NONE'
    
    try:
        
        res_idx = int_res_one_hot.index(1)
        real_idx = int_real_one_hot.index(1)
        #print('Result Index: {} | Expected Index: {}'.format(res_idx, real_idx))
        
        res_ = labels[res_idx]
        real_ = labels[real_idx]
        #print('Result Index: {} | Expected Index: {}'.format(res_, real_))
        
    except:
        print('NONE')    
    
    prediction = res_
    expected = real_
    correct = (prediction == expected)
    
    return prediction, expected, correct

# Run Inference On A Few Random Images

In [ ]:
data, _ = read_data('input', 1, 0) # read one image from each category

x = []
y = []

for feature, label in data:
    x.append(feature)
    y.append(label)
    
x = np.array(x) / 255
y = keras.utils.to_categorical(np.array(y))
    
for img, real in zip(x, y):
    img.resize(1, img_rows, img_cols, 3)
    prediction, expected, correct = inference(model, img, real)
    print('Result Index: {} | Expected Index: {}'.format(prediction, expected))

# Find Misclassified Samples
If no images are printed, there are no misclassified samples and that is good. 

In [ ]:
data, _ = read_data('input', 500, 0) # read 500 images from each category

x_ = []
y_ = []

for feature, label in data:
    x_.append(feature)
    y_.append(label)
    
x_ = np.array(x_) / 255
y_ = keras.utils.to_categorical(np.array(y_))
    
for img, real in zip(x_, y_):
    img.resize(1, img_rows, img_cols, 3)
    prediction, expected, correct = inference(model, img, real)
    if not correct:
        message = 'Model predicted "{}" but label is "{}"'.format(prediction, expected)
        print(message)
        plt.figure(figsize=(5,5))
        tmp = img.copy()[0]
        plt.imshow(tmp)
        plt.title(message)

# Save Model

In [ ]:
model.save('./path-classifier.h5')